In [1]:
import yapo as y
import pandas as pd
import numpy as np

## Asset Creation

The library can create an `asset` in several ways. The method `portfolio_asset` accepts parameters:

- either `name: str`, or `names: List[str]` of asset(s) to be created
- optional `start_period: str` and `end_period: str` to define date range. If nothing is provided then the library takes maximum possible range 
- optional `currency: str` to compute further activities. If no is provided then the library takes the default value for the financial symbol

In [2]:
asset_name = 'us/SPY'

In [3]:
def print_asset(asset):
    print('== {}'.format(asset.symbol.identifier_str))
    print(asset.currency)
    print(asset._period_min)

In [4]:
print_asset(y.portfolio_asset(name=asset_name))

== us/SPY
Currency(USD)
1993-01


In [5]:
print_asset(y.portfolio_asset(name=asset_name, start_period='2010-1'))

== us/SPY
Currency(USD)
2010-01


In [6]:
print_asset(y.portfolio_asset(name=asset_name, start_period='2010-1', end_period='2013-1'))

== us/SPY
Currency(USD)
2010-01


In [7]:
asset=y.portfolio_asset(name=asset_name, 
                        start_period='2010-1', end_period='2013-1', 
                        currency='usd')
print_asset(asset)

== us/SPY
Currency(USD)
2010-01


## Basic Activities

### Close

Every asset has the `close` values. The `close` method returns `TimeSeries` object that wraps `numpy` array with additional meta-information: `start_period`, `end_period`, and `TimeSeriesKind`

In [8]:
close = asset.close()
close

TimeSeries(start_period=2010-01, end_period=2013-01, kind=TimeSeriesKind.VALUES, values=[ 89.1311  91.9114  97.1072  98.6113  90.7734  86.0777  91.9568  87.8205
  95.685   99.3406  99.3427 105.9819 108.4512 112.2185 112.2317 115.4825
 114.1869 112.2609 110.0153 103.967   96.7496 107.3096 106.8735 107.99
 112.9979 117.9029 121.695  120.8825 113.6228 118.2381 119.6326 122.6297
 125.7384 123.4503 124.1531 125.2578 131.6699]

In [9]:
close.values

array([ 89.1311,  91.9114,  97.1072,  98.6113,  90.7734,  86.0777,
        91.9568,  87.8205,  95.685 ,  99.3406,  99.3427, 105.9819,
       108.4512, 112.2185, 112.2317, 115.4825, 114.1869, 112.2609,
       110.0153, 103.967 ,  96.7496, 107.3096, 106.8735, 107.99  ,
       112.9979, 117.9029, 121.695 , 120.8825, 113.6228, 118.2381,
       119.6326, 122.6297, 125.7384, 123.4503, 124.1531, 125.2578,
       131.6699])

In [10]:
close.start_period, close.end_period

(Period('2010-01', 'M'), Period('2013-01', 'M'))

### Return

Next thing we derive from the `Close` values is [the `Return`](https://okama.io/#/glossary?page=cumulative-return). The `get_return` produces `TimeSeries` object that has one-month shorter `start_period`

In [11]:
assert asset.get_return().start_period == asset.close().start_period + 1
asset.get_return()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.DIFF, values=[ 3.11933769e-02  5.65305283e-02  1.54890678e-02 -7.94827773e-02
 -5.17299121e-02  6.82999197e-02 -4.49809041e-02  8.95519839e-02
  3.82045253e-02  2.11393932e-05  6.68312820e-02  2.32992615e-02
  3.47372828e-02  1.17627664e-04  2.89650785e-02 -1.12190159e-02
 -1.68670837e-02 -2.00034028e-02 -5.49768987e-02 -6.94201045e-02
  1.09147738e-01 -4.06394209e-03  1.04469302e-02  4.63737383e-02
  4.34078863e-02  3.21629069e-02 -6.67652738e-03 -6.00558393e-02
  4.06194883e-02  1.17939987e-02  2.50525358e-02  2.53503026e-02
 -1.81973049e-02  5.69297928e-03  8.89788495e-03  5.11912232e-02]

`get_return` accepts parameters as follows:

- `kind: str` that determines the time series reduction. It should be one of `values` (by default), `cumulative` or `ytd` values
- `real: bool` that states if the return should be adjusted to corresponding inflation (`False` by default)

For example, the call as follows gets the real cumulative return for the period:

In [12]:
asset.get_return(kind='cumulative', real=True)

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.CUMULATIVE, values=[ 0.03093667  0.08476204  0.09965389  0.01146646 -0.03991957  0.02543747
 -0.0220382   0.06492044  0.10423045  0.1037891   0.17553717  0.19722394
  0.23273364  0.22097293  0.2483007   0.22851705  0.20909049  0.18385567
  0.11569387  0.03666862  0.15219564  0.1484825   0.1633506   0.21196686
  0.25903227  0.2897309   0.27726137  0.20196427  0.25262379  0.26946646
  0.29406832  0.32097914  0.29744559  0.31104364  0.32628083  0.39006435]

### CAGR

Asset supports computation of [CAGR](https://okama.io/#/glossary?page=compound-annual-growth-rate) activity:

In [13]:
asset.cagr()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.1389004]

The asset can also compute the real CAGR:

In [14]:
asset.cagr(real=True)

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.11603625]

### Risk

The Risk ([Standard Deviation](https://okama.io/#/glossary?page=standard-deviation)) is computed with `risk: TimeSeries` method. It accepts `period` parameter that is either `year` (by default) or `month`:

In [15]:
asset.risk()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.17108709]

In [16]:
asset.risk(period='year')

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.17108709]

In [17]:
asset.risk(period='month')

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.04318064]